In [ ]:
# TODO put into github repo
# TODO change the code into function 
# TODO add looping functionarity to be able to iterate multiple links
# TODO figure out how to deal with exceptions 
# TODO analyze the data and perform visualization

In [285]:
import requests
import praw
from praw.models import MoreComments 
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup
import json

In [286]:
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [287]:
import json 
json_file = "reddit_api_config.json"
json_key = "reddit_api"
with open(json_file) as f:
    data = json.load(f)
user_values = data[json_key]

reddit = praw.Reddit(
    client_id=user_values['client_id'],
    client_secret=user_values['client_secret'],
    user_agent=user_values['user_agent'],
)


In [289]:
# Test if the read only instance was created properly 
print(reddit.read_only)

True


In [290]:
# Select the subreddit 
subreddit = reddit.subreddit("Accounting").stream
# For this case, we have a specific url that we would like to work on 
# url = "https://www.reddit.com/r/Accounting/comments/o6c81e/pwc_2021_compensation_thread/"
url = "https://www.reddit.com/r/Accounting/comments/hb5u1h/pwc_2020_compensation_thread/"
# Create submission object bsaed on the url 
submission = reddit.submission(url=url)
file_name = submission.title.replace(" ", "_")
file_name = f"{file_name}.csv"

For this url, the breakdown of the category is as below 
1. Market/Office
2. Trust or Consulting Solutions and LOS/Vertical
3. CY Level -> FY22 Level (A1>A2, S1->S2, S3->M1, etc)
4. Rating
5. Old Salary -> New Salary
6. Bonus
7. Interesting notes on what RLs/RPs have told you related to future comp.
8. Anything else? (opinions on the cohort model for all LOS, opinions on the new equation, etc)

In [291]:
# create a dictionary to organize the submission comments
# category_list = [
#     "Location",
#     "Type",
#     "Change in level",
#     "Rating",
#     "Salary",
#     "Bonus",
#     "Notes",
#     "Other",
# ]
# numbering for the category list
# num_keys = range (1, len(category_list)+1)

# # create looup dictionary to identify category based on the number 
# lookup_dict = dict(zip(num_keys, category_list))

# # this dictionary will keep track of which items are responded
# check_dict = {i:False for i in num_keys}


In [292]:
# Attemp to create a category automatically 
# obtaining the body of the OP's text in html format
submission_text = submission.selftext_html
# create soup object 
soup = BeautifulSoup(submission_text, 'html.parser')
# the list is always in ordered list, or ol. hence find that 
category_list = soup.find("ol")
# we category_list.text will provide the str of category, hence split to make a list 
category_list = category_list.text.split("\n")
# remove the empty elements in the list 
category_list = list(filter(None,category_list))
# numbering for the category list
num_list = range(1, len(category_list)+1)
# create lookup dictionary to identify category based on the number 
lookup_dict = dict(zip(num_list, category_list))
# print(lookup_dict)

# this dictionary will keep track of which items are responded
check_dict = {i:False for i in num_list}

In [293]:
# using the defaultdict library, create a dictionary of list 
submission_dict = defaultdict(list)


In [294]:
# obtain the submission comments object 
# comments = submission.comments.replace_more(limit=None, threshold=0)
comments = submission.comments


In [295]:
for i in range(1,7):
    print(i)

1
2
3
4
5
6


In [296]:
# Need to call replace_more in order to avoid the Morecomments error
comments.replace_more(limit=None)
# for each comment in the comments object
for comment in comments: 
    print(f"worked on the {comment.body}")
    # obtain the comment body 
    content = comment.body
    # split the contents line by line  
    content = content.split("\n")
    # remove empty strings from the list 
    content = list(filter(None, content))
    # only select line that start with the numbers 
    content = [i for i in content if i[0].isnumeric()]
    # if the list is blank 
    if not content: 
        #skip that item
        continue
    # create list of numbers from dict keys
    key_list = str(list(lookup_dict.keys()))
    # for each line in the content, or the comment 
    for line in content:
        # the number is always the first character 
        heading_number = float(line[0])
        # obtain the category using the look up table 
        if check_dict[heading_number] == True: 
            continue
        category = lookup_dict[heading_number]
        check_dict[heading_number] = True
        # append the response based on the category key 
        submission_dict[category].append(line.strip()[2:].strip())
        # print(len(submission_dict[category]))

        # print(f"worked on the {line}")
    print(num_list)
    for number in num_list:
        if check_dict[number]:
            check_dict[number]=False
            # print(f"did respond {number}")
            # continue
        else:
            print(number)
            category = lookup_dict[number]
            # print("category")
            # print(f"before {submission_dict[category]}")
            submission_dict[category].append("")
            # print(f"after {submission_dict[category]}")
            check_dict[number]=False
            # print(f"did not respond {number}")
            # continue
    
    # we need to be careful because some people do not post response for all the question. Therefore, if they skip one or 

worked on the Stickying the [2019 PwC comp thread](https://www.reddit.com/r/Accounting/comments/c1ob0e/pwc_2019_compensation_thread/) for comparative purposes.
worked on the 1. NYC
2. Assurance
3. 2 (damn a lot of 1s in here)
4. A1 -> A2 
5. 61k -> .... 61k ahahahahhahahah
range(1, 8)
6
7
worked on the 1. NY Metro
2. FS
3. A2 -> S1
4. 1
5. 69500 -> 75900
6. 10% increase for promos to senior and manager, 5% to senior manager and director; no insight on bonuses
range(1, 8)
7
worked on the 1. Southeast
2. Core Assurance
3. A3 -> S1
4. 1
5. 59,300 -> 65,300
range(1, 8)
6
7
worked on the How many of you guys also plan on staying at PwC? That should be what the seventh question is referring to
worked on the [deleted]
worked on the BANW - SJ

Tax

A2 -> S1

1

72.5k - 81.9k

Don't think I truly deserved a 1 but heard there was almost no pushback from national on suggested tiers (e.g. no one suggested for a 1 got bumped down to 2). Probably consolation for decreased pay bands. Missed out on cl

In [297]:
for key in list(submission_dict.keys()):
    print(len(submission_dict[key]))

50
50
50
50
50
50
50


In [298]:
df = pd.DataFrame(submission_dict)
df.head()
df.to_csv(file_name)

In [299]:
%reset -f
# The above magic command reset the variable, -f denote no user confirmation 